<a href="https://colab.research.google.com/github/Shorubiga/Detection-and-Classification-of-Distributed-DoS-Attacks-using-Machine-Learning/blob/master/DDOS_CNN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
 #%tensorflow_version 2.x
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import numpy as np
np.random.seed(0)

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
#loading dataset
import pandas as pd
x = pd.read_csv("/content/drive/My Drive/DatasetCNNDDoS/clean_1.csv", engine='python')
y = pd.read_csv("/content/drive/My Drive/DatasetCNNDDoS/y.csv", engine='python')



In [5]:
y.head()

,Unnamed: 0,Label
0,0,2
1,1,2
2,2,2
3,3,2
4,4,2


In [6]:
y=y['Label']
x=x.drop(['Label'], axis=1)


In [7]:
x.head()

,Unnamed: 0,Source_IP,Source_Port,Destination_IP,Destination_Port,Protocol,Flow_Duration,Total_Length_of_Fwd Packets,Total_Length_of_Bwd Packets,Fwd Packet Length Mean,...,Packet Length Variance,SYN Flag Count,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,act_data_pkt_fwd,min_seg_size_forward,Inbound
0,0,2886729733,57177,3232248324,19971,6,188912,24,12,6.0,...,0.0,0,7.0,6.0,6.0,80,4,3,20,1
1,1,2886729733,31536,3232248324,23,6,183,12,12,6.0,...,0.0,0,7.5,6.0,6.0,40,2,1,20,1
2,2,2886729733,52312,3232248324,20889,6,168997,24,12,6.0,...,0.0,0,7.0,6.0,6.0,80,4,3,20,1
3,3,2886729733,44853,3232248324,1274,6,151583,24,12,6.0,...,0.0,0,7.0,6.0,6.0,80,4,3,20,1
4,4,2886729733,648,3232248324,35182,17,3,458,0,229.0,...,0.0,0,343.5,229.0,0.0,64,2,1,32,1


In [8]:
x.shape

(1048499, 27)

In [10]:
#normalizing the data
normalized_df=(x-x.mean())/x.std()
#normalized_df=preprocessing.normalize(x)

In [ ]:
normalized_df


array([[0.00000000e+00, 6.66117674e-01, 1.31936876e-05, ...,
        6.92254976e-10, 4.61503317e-09, 2.30751659e-10],
       [2.30751659e-10, 6.66117674e-01, 7.27698431e-06, ...,
        2.30751659e-10, 4.61503318e-09, 2.30751659e-10],
       [4.61503317e-10, 6.66117674e-01, 1.20710808e-05, ...,
        6.92254976e-10, 4.61503317e-09, 2.30751659e-10],
       ...,
       [2.41942184e-04, 6.66117655e-01, 8.21729708e-06, ...,
        2.30751652e-10, 4.61503304e-09, 2.30751652e-10],
       [2.41942415e-04, 6.66117655e-01, 8.21752783e-06, ...,
        2.30751652e-10, 4.61503304e-09, 2.30751652e-10],
       [2.41942646e-04, 6.66117655e-01, 8.21775858e-06, ...,
        2.30751652e-10, 4.61503304e-09, 2.30751652e-10]])

In [11]:
y.nunique()

8

In [12]:
y = pd.get_dummies(y,prefix=['Label'])
y.head()

,['Label']_0,['Label']_1,['Label']_2,['Label']_3,['Label']_4,['Label']_5,['Label']_6,['Label']_7
0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0


In [13]:
y.shape


(1048499, 8)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normalized_df, y, test_size=0.25, random_state=0)
y_train = np.array(y_train)

In [15]:
y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [16]:
print('xtrain={}, ytrain={}, xtest={}, ytest={}'.format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

xtrain=(786374, 27, 1), ytrain=(786374, 8), xtest=(262125, 27, 1), ytest=(262125, 8)


In [17]:
from keras import optimizers
sgd = optimizers.SGD(learning_rate=0.009, decay=1e-6, momentum=0.9, nesterov=True)

In [18]:
m2 = Sequential()
m2.add(Convolution1D(64, 3, activation="relu", input_shape=(27,1)))
m2.add(Convolution1D(32, 3, activation="relu"))
m2.add(MaxPooling1D(pool_size=2))
m2.add(Flatten())
m2.add(Dense(8))


m2.compile(loss="mean_absolute_error", optimizer="adam", metrics=['accuracy'])
m2.fit(X_train, y_train, epochs=15, batch_size=5000, validation_data=(X_test, y_test))

Epoch 1/15
158/158 [==============================] - 13s 18ms/step - loss: 0.1064 - accuracy: 0.7840 - val_loss: 0.0840 - val_accuracy: 0.9048
Epoch 2/15
158/158 [==============================] - 3s 19ms/step - loss: 0.0570 - accuracy: 0.9407 - val_loss: 0.0388 - val_accuracy: 0.9691
Epoch 3/15
158/158 [==============================] - 2s 15ms/step - loss: 0.0351 - accuracy: 0.9751 - val_loss: 0.0317 - val_accuracy: 0.9787
Epoch 4/15
158/158 [==============================] - 2s 15ms/step - loss: 0.0304 - accuracy: 0.9794 - val_loss: 0.0288 - val_accuracy: 0.9799
Epoch 5/15
158/158 [==============================] - 2s 16ms/step - loss: 0.0273 - accuracy: 0.9799 - val_loss: 0.0261 - val_accuracy: 0.9803
Epoch 6/15
158/158 [==============================] - 2s 15ms/step - loss: 0.0247 - accuracy: 0.9802 - val_loss: 0.0229 - val_accuracy: 0.9807
Epoch 7/15
158/158 [==============================] - 2s 16ms/step - loss: 0.0226 - accuracy: 0.9808 - val_loss: 0.0217 - val_accuracy: 0.981

In [19]:
#save to folder
m2.save("/content/drive/My Drive/DatasetCNNDDoS/final_cnn_m2.h5")

In [20]:
y_pred = m2.predict(X_test)

8192/8192 [==============================] - 12s 1ms/step


In [21]:
a = (y_pred > 0.5)
b = (y_test > 0.5) 

In [22]:
b=np.argmax(y_test, axis=1)

In [23]:
a=np.argmax(y_pred, axis=1)
a[2]

4

In [24]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print('Convolution Neural Network')

print('Accuracy: %f' % (accuracy_score(a, b)*100))
print("Confusion Matrix =\n", metrics.confusion_matrix(b, a, labels=None, 
                                              sample_weight=None))
print("Recall =", metrics.recall_score(b, a, labels=None, 
                                             pos_label=1, average='weighted', 
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(b, a, 
                                                                 labels=None, 
                                                                 target_names=None, 
                                                                 sample_weight=None, 
                                                                 digits=2, 
                                                                 output_dict=False))

print("F1 Score = ",f1_score(a, b, average='macro'))

Convolution Neural Network
Accuracy: 98.721221
Confusion Matrix =
 [[14115   123     3    10    15     0     0     0]
 [    1 49892     3    15     5     0     0     0]
 [    7    15 49676   210     5     0     0     0]
 [    3     4  1232 48925    11     0     0     0]
 [  134     4     0    77 46348     0    61     0]
 [  623     0     0     0     0     0    59     0]
 [    5     1     0   240    11     0 49817     0]
 [   33     0     0    11    13     0   418     0]]
Recall = 0.9872122079160706


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report =
               precision    recall  f1-score   support

           0       0.95      0.99      0.97     14266
           1       1.00      1.00      1.00     49916
           2       0.98      1.00      0.99     49913
           3       0.99      0.98      0.98     50175
           4       1.00      0.99      1.00     46624
           5       0.00      0.00      0.00       682
           6       0.99      0.99      0.99     50074
           7       0.00      0.00      0.00       475

    accuracy                           0.99    262125
   macro avg       0.74      0.74      0.74    262125
weighted avg       0.98      0.99      0.99    262125

F1 Score =  0.7401440795612984


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
